In [14]:
!pip install ipywidgets


# 1. Mount Drive & Imports
from google.colab import drive
drive.mount('/content/drive')

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# 2. Connect to DB and load tables
conn = sqlite3.connect('imdb_filtered.db')

# Load CSVs to SQLite (adjust paths as needed)
df_movies = pd.read_csv('/content/drive/MyDrive/IMDB_Data/filtered/merged_movies.csv')
df_movies.to_sql('movies', conn, if_exists='replace', index=False)

df_principals = pd.read_csv('/content/drive/MyDrive/IMDB_Data/filtered/principals_filtered.csv')
df_principals.to_sql('filtered_principals', conn, if_exists='replace', index=False)

df_names = pd.read_csv('/content/drive/MyDrive/IMDB_Data/filtered/names_filtered.csv')
df_names.to_sql('filtered_names', conn, if_exists='replace', index=False)

# Create worst_200_movies temp table
conn.execute("""
CREATE TEMP TABLE worst_200_movies AS
SELECT * FROM movies
ORDER BY averageRating ASC, numVotes DESC
LIMIT 200
""")

# 3. Query top 50 frequent people
query_frequent = """
SELECT
    n.primaryName,
    COUNT(*) AS appearances
FROM worst_200_movies w
JOIN filtered_principals p ON w.tconst = p.tconst
JOIN filtered_names n ON p.nconst = n.nconst
GROUP BY n.primaryName
ORDER BY appearances DESC
LIMIT 50;
"""
df_frequent = pd.read_sql(query_frequent, conn)

# 4. Query top weighted culprits
query_weighted = """
WITH role_weights AS (
  SELECT 'producer' AS role, 4 AS weight UNION ALL
  SELECT 'director', 3 UNION ALL
  SELECT 'writer', 2 UNION ALL
  SELECT 'actor', 1 UNION ALL
  SELECT 'actress', 1
),
merged AS (
  SELECT
    n.primaryName,
    LOWER(p.category) AS role,
    w.numVotes,
    COUNT(*) as appearances,
    rw.weight
  FROM worst_200_movies w
  JOIN filtered_principals p ON w.tconst = p.tconst
  JOIN filtered_names n ON p.nconst = n.nconst
  JOIN role_weights rw ON LOWER(p.category) = rw.role
  GROUP BY n.primaryName, p.category
),
scored AS (
  SELECT
    primaryName,
    SUM(appearances * weight * numVotes) AS weighted_score
  FROM merged
  GROUP BY primaryName
)
SELECT primaryName, weighted_score
FROM scored
ORDER BY weighted_score DESC
LIMIT 50;
"""
df_weighted = pd.read_sql(query_weighted, conn)

# 5. Find weighted culprits NOT in frequent top 50
diff_df = df_weighted[~df_weighted.primaryName.isin(df_frequent.primaryName)]

# 6. Merge for plotting: appearances from frequent (if any), else zero
appearances_map = df_frequent.set_index('primaryName')['appearances'].to_dict()
diff_df['appearances'] = diff_df['primaryName'].map(appearances_map).fillna(0).astype(int)

# 7. Plot lollipop chart comparing appearances vs weighted score
diff_df_sorted = diff_df.sort_values('weighted_score')

plt.figure(figsize=(12,8))
plt.hlines(y=diff_df_sorted['primaryName'], xmin=0, xmax=diff_df_sorted['weighted_score'], color='red', alpha=0.6, linewidth=4)
plt.plot(diff_df_sorted['weighted_score'], diff_df_sorted['primaryName'], "o", color='crimson', label='Weighted Score', markersize=8)
plt.plot(diff_df_sorted['appearances'], diff_df_sorted['primaryName'], "o", color='blue', label='Appearances', markersize=5)

plt.xlabel('Score / Appearances')
plt.title('Weighted Culprits NOT in Top 50 Frequent People\nRed=Weighted Score, Blue=Appearances')
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# 8. Interactive widget to explore worst projects for a selected culprit
role_weights = {
    'producer': 4,
    'director': 3,
    'writer': 2,
    'actor': 1,
    'actress': 1
}

def show_culprit_projects(name):
    clear_output(wait=True)
    display(selector)
    sql = f"""
    WITH role_weights AS (
      SELECT 'producer' AS role, 4 AS weight UNION ALL
      SELECT 'director', 3 UNION ALL
      SELECT 'writer', 2 UNION ALL
      SELECT 'actor', 1 UNION ALL
      SELECT 'actress', 1
    ),
    culprit_movies AS (
      SELECT
        n.primaryName,
        w.tconst,
        t.primaryTitle,
        t.startYear,
        w.averageRating,
        w.numVotes,
        p.category,
        rw.weight,
        rw.weight * w.numVotes AS impact_score
      FROM worst_200_movies w
      JOIN filtered_principals p ON w.tconst = p.tconst
      JOIN filtered_names n ON p.nconst = n.nconst
      JOIN role_weights rw ON LOWER(p.category) = rw.role
      JOIN movies t ON w.tconst = t.tconst
      WHERE n.primaryName = ?
    )
    SELECT
      primaryTitle,
      startYear,
      averageRating,
      numVotes,
      category AS role,
      impact_score
    FROM culprit_movies
    ORDER BY impact_score DESC
    LIMIT 5;
    """
    df_projects = pd.read_sql(sql, conn, params=(name,))
    if df_projects.empty:
        print(f"No data found for {name}.")
    else:
        print(f"Top bad projects for {name}:")
        display(df_projects)

# Widget for culprit selection
selector = widgets.Dropdown(
    options=diff_df['primaryName'].tolist(),
    description='Select Culprit:',
    layout=widgets.Layout(width='70%')
)
selector.observe(lambda change: show_culprit_projects(change['new']), names='value')

display(selector)


Dropdown(description='Select Culprit:', index=4, layout=Layout(width='70%'), options=('Jared LeBoff', 'Marc Pl…

Top bad projects for Jason Friedberg:


,primaryTitle,startYear,averageRating,numVotes,role,impact_score
0,Disaster Movie,2008.0,1.9,96517,producer,386068
1,Disaster Movie,2008.0,1.9,96517,director,289551
2,Disaster Movie,2008.0,1.9,96517,writer,193034
